In [1]:
import csv
import datetime


In [2]:
## read course Data set
courseList = []
with open("./MOOC_data/xtData/(cleand)ClassCentral_course.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courseList.append(row)
        
print(len(courseList))

3153


In [3]:
## read review Data set
reviewList = []
with open("./MOOC_data/xtData/(Date_Adjusted)ClassCentral_review_renewed.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        reviewList.append(row)
print(len(reviewList))

20903


In [4]:
userList = []
with open("./MOOC_data/xtData/classCentral_userData.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        flag = False
        for index, user in enumerate(userList):
            
            if user['user_url'] == row['user_url']:
                userList[index]['courses'].append(row['course_url'])
                flag = True
                
        if flag == False:
            userList.append(dict({
                'user_url': row['user_url'],
                'courses': [row['course_url']]
            
            }))
        
len(userList)
print(userList[2])

{'user_url': '/u/143184', 'courses': ['/mooc/2353/coursera-chinese-for-beginners']}


In [5]:
courseList[0]

{'course_length (week)': '13',
 'description': 'Class Central Course Rank #1 in Subjects > Education & Teaching > Test Prep This test preparation course, developed by the experts who create, administer and score the TOEFL?test, will help English language learners improve their skills. The TOEFL?test is the world’s most trusted and widely accepted English-language assessment. It has helped millions of people achieve their dreams to study, work or live abroad. More than 9,000 institutions of higher education, government agencies and organizations worldwide accept TOEFL?scores for making important decisions. This course will help you understand what you can do to achieve your best TOEFL test score. Instructors will guide you through each section (Reading, Listening, Speaking and Writing) and, using archived past test questions, will explain the kinds of questions you can expect. You will receive valuable advice, including how to register for the test, how it is scored and how to prepare f

<h3> Generate directed course network edgelist </h3>

In [7]:
## 사용자, 사용자의 수강리스트 만들기.

reviewers =[]
for review in reviewList:
    
    
    # reviewer id가 없을 경우 continue
    if review['reviewer_url'] == "null":
        continue
        
    
    # reviewers list 를 돌면서 course 추가
    flag = False
    for index, re in enumerate(reviewers):
        
        
        if re['reviewer'] == review['reviewer_url']:
            reviewers[index]['courses'].append(dict({
                    'date': review['review_date'],
                    'course': review['url']
                }))
            flag = True
            
            
    
    if flag == False:
        reviewers.append(dict({
            'reviewer': review['reviewer_url'],
            'courses': [dict({
                'date': review['review_date'],
                'course': review['url']
            })]
            
        }))
        

In [8]:
final_t1_edges = []
final_t2_edges = []

final_t1_nodes = []
final_t2_nodes = []


In [10]:

# course directed edge list 생성
for re in reviewers:
    t1_edges = []
    t2_edges = []

    t1_nodes = []
    t2_nodes = []
       
    for co in re['courses']:

               
        date = datetime.datetime.strptime(co['date'], "%Y-%m-%d") 
        
        if date.year == 2013:

            # T1: 
            if date.month > 6 and date.month <= 12:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
        
        elif date.year == 2014:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T2: 
            else:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                

        elif date.year == 2015:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T3: 15.7~12
            else:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2016:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T4
            else:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2017:
            
            if date.month > 0 and date.month < 7:
            
                t2_nodes.append(dict({
                        'course': co['course'],
                        'date': co['date']           
                    }))
    
    # 날짜별 sorting
    t1_nodes.sort(key=lambda item:item['date'], reverse=False)
    t2_nodes.sort(key=lambda item:item['date'], reverse=False)
        
    # 같은 vertex time에 있는 노드가 두개 이상인지 체크, 2개 이상일 경우 date 비교 후 edgelist에 추가
        
    final_t1_edges.extend(get_edgelist(t1_nodes, re['reviewer']))
    final_t2_edges.extend(get_edgelist(t2_nodes, re['reviewer']))
        
    final_t1_nodes.extend(t1_nodes)
    final_t2_nodes.extend(t2_nodes)
    


In [9]:
def get_edgelist(nodes, reviewer_id):
    temp_edges = []
    base_str = "https://www.class-central.com"
    for i in range(0, len(nodes)-1):
        
        
        if nodes[i]['date'] == nodes[i+1]['date']:
            ## review date가 같을 경우 양방향 edge, i <=> i+1
            
            ## user List에서 url 매칭 이후 순서 비교해서 edge 형성
            for user in userList:
                
                # 사용자가 같을 경우, 사용자의 강의리스트를 돌면서 index 부여.
                if user['user_url'] == reviewer_id:
                    first_index = 999
                    second_index = 999
                    for index, co in enumerate(user['courses']):
                        if base_str+co == nodes[i]['course']:
                            first_index = index
                        elif base_str+co == nodes[i+1]['course']:
                            second_index = index
                    
                    # index 부여 끝.
                    
                    # index가 초기값과 같으면 양방향 엣지 형성.
                    if first_index == 999 or second_index == 999:
                        temp_edges.append(dict({
                            'from': nodes[i]['course'],
                            'to': nodes[i+1]['course']
                        }))

                        temp_edges.append(dict({
                            'from': nodes[i+1]['course'],
                            'to': nodes[i]['course']
                        }))
                    else:
                        # 그렇지 않다면 index 비교, 작은쪽이 from
                        if first_index < second_index:
                            temp_edges.append(dict({
                                'from': nodes[i]['course'],
                                'to': nodes[i+1]['course']
                            }))
                        else:
                            temp_edges.append(dict({
                                'from': nodes[i+1]['course'],
                                'to': nodes[i]['course']
                            }))
                    
                else:
                    pass
                   
            
            
#             print ("make edge", nodes[i+1]['course'], nodes[i]['course'])
            
        else:
            ## review date가 다를 경우는 먼저 있는 녀석이 앞에 있음, i => i+1
            temp_edges.append(dict({
                'from': nodes[i]['course'],
                'to': nodes[i+1]['course']
            }))
#             print ("make edge", nodes[i+1]['course'], nodes[i]['course'])
            
    return temp_edges

    

In [11]:
# write edge list

with open('./MOOC_data/xtData/xt_1year(class_central)/edgelist_t1-3(directed).csv', 'w', newline = '') as write_file:
    
    fieldnames = ['from', 'to']
    
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in final_t1_edges:
        dict_writer.writerow(row)


In [12]:
# write node list

with open('./MOOC_data/xtData/xt_1year(class_central)/nodelist_t1-3(directed).csv', 'w', newline = '') as write_file:
    
    fieldnames = ['course', 'date']
    
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in final_t1_nodes:
        dict_writer.writerow(row)
        


In [14]:
len(final_t1_edges)

2497

<h3> (Currently not used) Try to construct network with igraph-python </h3>

In [60]:
import igraph

In [152]:
dict_time1 = {
    'time': 1,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time2 = {
    'time': 2,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time3 = {
    'time': 3,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time4 = {
    'time': 4,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time5 = {
    'time': 5,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time6 = {
    'time': 6,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time7 = {
    'time': 7,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time8 = {
    'time': 8,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}
dict_time9 = {
    'time': 9,
    'edges': [],
    'vertexType': [],
    'vertexList':[]
}

In [113]:
temp_list =[]
temp_list.append(tuple({"heungseok", "park"}))
temp_list.append(tuple({"heungseok", "park"}))
for review in reviewList:
    temp_list.append(tuple({review['reviewer_url'], review['url']}))
temp_list[:] = []

In [136]:
g = igraph.Graph.Bipartite(dict_time3['vertexType'], dict_time3['edges'])

TypeError: only numbers, vertex names or igraph.Vertex objects can be converted to vertex IDs

In [122]:
########################### TUPLE ver. #####################
## time iteration & construct network
bipartite_list = []


# review 돌면서 reviewer url이 not null 인 dict(course_url, reviewer_url) list of tuple만들기
for review in reviewList:
    if review['reviewer_url'] != 'null':

        date = datetime.datetime.strptime(review['review_date'], "%Y-%m-%d") 
        if date.year == 2013:

            # T1: 13.1~6
            if date.month > 0 and date.month < 7:
                dict_time1['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time1['vertexList']:
                    dict_time1['vertexList'].append(review['reviewer_url'])
                    dict_time1['vertexType'].append(0)
                if review['url'] not in dict_time1['vertexList']:
                    dict_time1['vertexList'].append(review['url'])
                    dict_time1['vertexType'].append(1)
            # T2: 13.7~12
            else:
                dict_time2['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time2['vertexList']:
                    dict_time2['vertexList'].append(review['reviewer_url'])
                    dict_time2['vertexType'].append(0)
                if review['url'] not in dict_time2['vertexList']:
                    dict_time2['vertexList'].append(review['url'])
                    dict_time2['vertexType'].append(1)

        elif date.year == 2014:
            # T3: 14.1~6
            if date.month > 0 and date.month < 7:
                dict_time3['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time3['vertexList']:
                    dict_time3['vertexList'].append(review['reviewer_url'])
                    dict_time3['vertexType'].append(0)
                if review['url'] not in dict_time3['vertexList']:
                    dict_time3['vertexList'].append(review['url'])
                    dict_time3['vertexType'].append(1)
            # T4: 14.7~12
            else:
                dict_time4['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time4['vertexList']:
                    dict_time4['vertexList'].append(review['reviewer_url'])
                    dict_time4['vertexType'].append(0)
                if review['url'] not in dict_time4['vertexList']:
                    dict_time4['vertexList'].append(review['url'])
                    dict_time4['vertexType'].append(1)

        elif date.year == 2015:
            # T5: 15.1~6
            if date.month > 0 and date.month < 7:
                dict_time5['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time5['vertexList']:
                    dict_time5['vertexList'].append(review['reviewer_url'])
                    dict_time5['vertexType'].append(0)
                if review['url'] not in dict_time5['vertexList']:
                    dict_time5['vertexList'].append(review['url'])
                    dict_time5['vertexType'].append(1)
            # T6: 15.7~12
            else:
                dict_time6['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time6['vertexList']:
                    dict_time6['vertexList'].append(review['reviewer_url'])
                    dict_time6['vertexType'].append(0)
                if review['url'] not in dict_time6['vertexList']:
                    dict_time6['vertexList'].append(review['url'])
                    dict_time6['vertexType'].append(1)
        elif date.year == 2016:
            # T7: 16.1~6
            if date.month > 0 and date.month < 7:
                dict_time7['edges'].append(tuple({review['reviewer_url'],review['url']}))
                if review['reviewer_url'] not in dict_time7['vertexList']:
                    dict_time7['vertexList'].append(review['reviewer_url'])
                    dict_time7['vertexType'].append(0)
                if review['url'] not in dict_time7['vertexList']:
                    dict_time7['vertexList'].append(review['url'])
                    dict_time7['vertexType'].append(1)
            # T8: 16.7~12
            else:
                dict_time8['edges'].append(tuple({ review['reviewer_url'],review['url'] }))
                if review['reviewer_url'] not in dict_time8['vertexList']:
                    dict_time8['vertexList'].append(review['reviewer_url'])
                    dict_time8['vertexType'].append(0)
                if review['url'] not in dict_time8['vertexList']:
                    dict_time8['vertexList'].append(review['url'])
                    dict_time8['vertexType'].append(1)
        elif date.year == 2017:
            # T9: 17.1~6
            if date.month > 0 and date.month < 7:
                dict_time9['edges'].append(tuple({ review['reviewer_url'], review['url'] }))
                if review['reviewer_url'] not in dict_time9['vertexList']:
                    dict_time9['vertexList'].append(review['reviewer_url'])
                    dict_time9['vertexType'].append(0)
                if review['url'] not in dict_time9['vertexList']:
                    dict_time9['vertexList'].append(review['url'])
                    dict_time9['vertexType'].append(1)



